## <center style="color:blue;">**SparkAttriNet**</center>

Dans le secteur bancaire, anticiper la perte de clients est essentiel pour réduire le taux d’attrition et renforcer la fidélisation. 

Ce projet exploite la puissance de PySpark pour analyser de grands volumes de données, MLlib pour entraîner un modèle prédictif, MongoDB pour stocker les données transformées, et Streamlit pour visualiser les résultats et faciliter la prise de décision.

<br>

### <span style="color:green;">**Configuration et Initialisation du Pipeline Spark :**</span>

#### <span style="color:orange;">**1. Apache Spark :**</span>

##### **1.1. Définition :**

``Apache Spark`` est un framework open-source conçu pour le traitement distribué de données à grande échelle.
Il permet d’exécuter des calculs complexes sur des volumes massifs de données en les répartissant sur plusieurs machines d’un cluster.

``Spark`` fonctionne en mémoire (in-memory computing), ce qui le rend beaucoup plus rapide que les systèmes traditionnels comme Hadoop MapReduce.

##### **1.2. A quoi sert ?**

``Spark`` sert principalement à analyser, transformer et traiter de grandes quantités de données de manière efficace et parallèle.
Il est utilisé pour des tâches variées :

- **Traitement de données massives (Big Data) :** nettoyage, agrégation, transformation de fichiers volumineux.

- **Analyse exploratoire et statistique** à grande échelle.

- **Machine Learning** via la bibliothèque intégrée ``MLlib``, pour entraîner des modèles sur d’énormes datasets.

- **Traitement de flux en temps réel** (avec ``Spark Streaming``) pour suivre des événements en continu.

En résumé, ``Spark`` est une plateforme unifiée qui permet de manipuler, analyser et modéliser des données massives, tout en offrant des performances élevées grâce à son exécution distribuée.

#### <span style="color:orange;">**2. Créer une session Spark optimisée :**</span>

La ``SparkSession`` est le point d’entrée principal pour interagir avec ``Spark``.

C’est le cerveau de chaque application ``Spark`` :

elle permet de créer des DataFrames, d’exécuter des requêtes SQL, de lire et écrire des fichiers (CSV, Parquet, JSON, etc.), et de configurer un cluster.

```js
spark = (
    SparkSession.builder
        .appName("AttriSpark - Client Attrition Prediction")
        .master("local[14]")
        .config("spark.driver.memory", "24g")
        .config("spark.executor.memory", "24g")
        .config("spark.sql.shuffle.partitions", "32")
        .config("spark.memory.fraction", "0.7")
        .config("spark.memory.storageFraction", "0.5")
        .config("spark.sql.adaptive.enabled", "true")
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
        .getOrCreate()
)
```

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkAttriNet - Prédiction de l'Attrition Client Bancaire") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

#### <span style="color:orange;">**3. Vérifier la version de Spark. :**</span>

In [2]:
print("Spark Session Créée avec Succès !")
print(f"Version Spark : {spark.version}")

Spark Session Créée avec Succès !
Version Spark : 4.0.1


<br>

### <span style="color:green;">**Chargement et Inspection des Données :**</span>

#### <span style="color:orange;">**1. Charger les données :**</span>

In [4]:
df = spark.read.csv(
    "../data/raw/data.csv",
    header=True,
    inferSchema=True
)

print("Données Chargées avec Succès !")

Données Chargées avec Succès !


#### <span style="color:orange;">**2. Afficher le Schéma des Colonnes :**</span>

In [5]:
df.printSchema() 

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



#### <span style="color:orange;">**3. Afficher un Extrait des Données :**</span>

In [6]:
df.show(5)

+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|    Boni|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93826.63|